In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
images = np.load("images.npy").reshape(18000, 150*150)
labels = np.load("labels.npy")
tot = np.append(images, labels, axis=1)
tot = tot.astype("float32")
np.random.shuffle(tot)
images, labels = tot[:,:22500].reshape(18000, 150, 150), tot[:,22500:]
images = images/255
labels = labels[:,0] + labels[:,1]/60
trainx, trainy = images[:int(18000*0.8)], labels[:int(18000*0.8)]
testx, testy = images[int(18000*0.8):], labels[int(18000*0.8):]

In [ ]:
model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(1, (5, 5), strides=(1, 1), activation="relu", input_shape=(150, 150, 1)))
model.add(keras.layers.Flatten(input_shape=(150, 150)))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(50, activation="relu"))
model.add(keras.layers.Dense(1, activation="relu"))
model.summary()

model.compile(
    loss="mean_squared_error",
    optimizer="Adam",
    metrics=[keras.metrics.MeanSquaredError()]
)

In [ ]:
history = model.fit(trainx, trainy, epochs=1)
model.save("task2model")

In [ ]:
model = keras.models.load_model("./task2model")
print(model.get_weights())
predictions = model.predict(testx[:100])
print(predictions)
for i in range(100):
    print(predictions[i])
    print(labels[i])